In [ ]:
import collections.abc
#hyper needs the four following aliases to be done manually.\n",
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

In [1]:
from os import path
import os
import numpy as np
import pandas as pd
import pickle
import mne
import matplotlib.pyplot as plt

In [ ]:
def from_pickle(path):
    with open(path, 'rb') as f:
        obj = pickle.load(f)
    return obj

In [2]:
path_raw_EEG = 'C:\\\\Users\\\\chedl\\\\OneDrive\\\\Bureau\\\\EMSE\\\\3A\\\\PE\\\\privateData\\\\EEG_MAHNOB_Dictinary.p'
path_raw_data = 'C:\\\\Users\\\\chedl\\\\OneDrive\\\\Bureau\\\\EMSE\\\\3A\\\\PE\\\\privateData\\\\Metadata_dict.p'
all_eeg = from_pickle(path_raw_EEG)
meta = from_pickle(path_raw_data)
labels = from_pickle(path.join(path_raw_data, "Label_Dict.p"))

547

In [ ]:
EEG_channels = ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3',
                'P7', 'PO3', 'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6',
                'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2']

In [ ]:
path_raw_data = "C:\\Users\\chedl\\OneDrive\\Bureau\\EMSE\\3A\\PE\\privateData"
# Merge the two dictionaries
merged_dict = {}
for subject_id, session_ids in meta.items():
    for session_id in session_ids['sessionId']:
    # Get the recordings for this session\n",
        recordings = all_eeg.get(session_id, [])
    # Update the merged dictionary with the participant ID, session ID, and recordings
        merged_dict.update({(subject_id, session_id): recordings})
    # Print the merged dictionary
#print(meta)

In [ ]:
path = 'C:\\\\Users\\\\chedl\\\\OneDrive\\\\Bureau\\\\EMSE\\\\3A\\\\PE\\\\essai'
# Set the path of the directory where you want to create the new directories
# Create 30 directories with names corresponding to their numbers in the specified directory\n",
for i in range(1, 31):
    dir_name = f'{i}'
    os.mkdir(os.path.join(path, dir_name))

In [ ]:
# Iterate through the subject IDs and their corresponding data dictionaries\n",
for subject_id, subject_session in meta.items():
    # Create the path to the subject directory
    subject_dir_path = os.path.join(path, str(subject_id))
    # Open the subject directory
    os.startfile(subject_dir_path)

    # Iterate through the session IDs and their corresponding data dictionaries for the current subject
    #for session_id, session_data in all_eeg.items():
    for session in subject_session['sessionId']:
        #Create the path to the session directory
        #session_dir_path = os.path.join(subject_dir_path, str(session))
        # Open the session directory
        #os.startfile(session_dir_path)
        session_data = all_eeg[session]
        session_data = session_data.reshape(32, -1)
        # Create a Raw object from the numpy array
        raw = mne.io.RawArray(session_data, info=mne.create_info(ch_names= EEG_channels, sfreq=1000))
        # Create the filename for the EDF file
        edf_filename = f"recording_{session}.raw.fif"
        # Write the Raw object to an EDF file in the current session's directory
        edf_path = os.path.join(subject_dir_path, edf_filename)
        raw.save(edf_path, overwrite=True)

In [ ]:
for i in range(1, 31):
    # Construct the path to the current subdirectory
    subdir = os.path.join(path, str(i))
    # Check if the subdirectory exists
    if os.path.exists(subdir) and os.path.isdir(subdir):
    # Loop through all the files in the subdirectory
        for file in os.listdir(subdir):
            # Check if the file is a .raw.fif file
            if file.endswith('.raw.fif'):
                # Construct the full path to the .raw.fif file
                raw_file = os.path.join(subdir, file)

                # Construct the full path to the .edf file
                edf_file = os.path.join(subdir, file[:-8] + '.edf')

                # Load the .raw.fif data
                raw = mne.io.read_raw_fif(raw_file)
                # Save the data as .edf, overwriting the existing file
                raw.save(edf_file, overwrite=True)

                # Delete the original .raw.fif file
                os.remove(raw_file)

In [ ]:
eeg = all_eeg[2][2]
eeg_data = all_eeg[2]
eeg_channels = EEG_channels
# Create the plot\n",
fig, ax = plt.subplots()
ax.plot(eeg)
# Set up the figure and axes
fig, axes = plt.subplots(nrows=len(eeg_channels), figsize=(200, 200))
# Set the plot title and axis labels\n",
ax.set_title("EEG Signal")
ax.set_xlabel("Time (samples)")
ax.set_ylabel("Voltage (uV)")
# Loop through each channel and plot the corresponding EEG data\n",
for i, channel in enumerate(eeg_channels):
    axes[i].plot(eeg_data[:, i])
    axes[i].set_title(channel)
# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
from dn3.configuratron import ExperimentConfig, DatasetConfig
from dn3.trainable.processes import StandardClassification
from dn3.trainable.models import TIDNet
# Since we are doing a lot of loading, this is nice to suppress some tedious information\n",
import mne
mne.set_log_level(False)

In [ ]:
config_filename = 'privateEEG_config.yml'
experiment = ExperimentConfig(config_filename)
ds_config = experiment.datasets['privateEEG']
dataset = ds_config.auto_construct_dataset()

In [ ]:
def make_model_and_process():
    tidnet = TIDNet.from_dataset(dataset)
    return StandardClassification(tidnet, cuda=experiment.use_gpu, learning_rate=ds_config.train_params.lr)

In [ ]:
results = list()
for training, validation, test in dataset.lmso(ds_config.train_params.folds):
    process = make_model_and_process()
    process.fit(training_dataset=training, validation_dataset=validation)
    results.append(process.evaluate(test)['Accuracy'])
    print(results)
    print("Average accuracy: {:.2%}".format(sum(results)/len(results)))